## Programación Lineal
### Coincidencia de Flujos de Caja
Si tenemos varias alternativas de financiación y disponemos de una planificación de los flujos de caja necesarios para un proyecto, podemos construir un problema de programación lineal para hacer coincidir los flujos de caja con el efectivo disponible en cada momento

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp

### Descripción 
Un proyecto de **5 años** tiene los siguientes flujos de caja estimados (en millones)

| Año | y1 | y2 | y3 | y4 | y5 |
|----:|---:|---:|---:|---:|---:|
|Flujos Netos | -5.2 | 0.4 | -1.5 | 3.7 | 2.8 |

Para el proyecto podemos financiarnos de la siguiente forma.
- Prestamos bancarios que se pagan al final de año a un interés del 0.03%
- Emitir bonos a 2 años con cupón anual del 0.02%
- El efectivo sobrante recibe un 0.005% anual invertido en mercados monetarios

### Modelo

In [ ]:
# años
n=5

# flujos de caja
cf = np.array([-5.2, 0.4, -1.5, 2.7, 7.8 ])
cf

In [ ]:
r_b = 0.03     #interes bancario
r_c = 0.02     #cupon bono
r_z = 0.005    #interes ahorro

In [ ]:
X = cp.Variable(5)
Y = cp.Variable(5)
Z = cp.Variable(5) 

In [ ]:
objetivo = cp.Maximize(Z[4])

In [ ]:
c_y1 =    X[0] + Y[0] - Z[0]                                         == -cf[0]
c_y2 =    X[1] + Y[1] - Z[1]  -X[0]*(1+r_b) -Y[0]*r_c + Z[0]*1.005   == -cf[1] 

In [ ]:
constraints = [X >= 0, Y>=0, Z>=0]
constraints.append(c_y1)
constraints.append(c_y2)

In [ ]:
for i in range(2,5):
    c_yi = X[i] + Y[i] - Z[i] - X[i-1]*(1+r_b) - Y[i-2]*(1+r_c) - Y[i-1]*r_c + Z[i-1]*(1+r_z) == -cf[i] 
    constraints.append(c_yi)

In [ ]:
constraints.append(X[-1] == 0)
constraints.append(Y[-2:] == 0)

In [ ]:
constraints

In [ ]:
prob = cp.Problem(objetivo, constraints)

In [ ]:
prob.solve(verbose=True)

In [ ]:
X.value.round(3)

In [ ]:
Y.value.round(3)

In [ ]:
Z.value

In [ ]:
xs = np.round(X.value,3)
ys = np.round(Y.value,3)
zs = np.round(Z.value,3)

In [ ]:
pd.DataFrame({
    'bancos':xs,
    'bonos':ys,
    'cash':zs,
    'cash_flow':cf
})